In [7]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [8]:
!kaggle datasets download -d datasnaek/youtube-new

##new method first kaggle.json file upload karvi pacchi enviorment set kar vo pachi download karvi


 97% 195M/201M [00:03<00:00, 69.7MB/s]
100% 201M/201M [00:03<00:00, 65.9MB/s]


In [9]:
!kaggle datasets download -d biaiscience/dogs-vs-cats

 98% 801M/817M [00:16<00:00, 26.0MB/s]
100% 817M/817M [00:16<00:00, 52.6MB/s]


In [10]:
!!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

["Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /content/kaggle.json'",
 'Downloading dogs-vs-cats-redux-kernels-edition.zip to /content',
 '',
 '  0% 0.00/814M [00:00<?, ?B/s]',
 '  1% 5.00M/814M [00:00<00:45, 18.7MB/s]',
 '  1% 9.00M/814M [00:00<00:44, 19.1MB/s]',
 '  3% 28.0M/814M [00:00<00:31, 26.1MB/s]',
 '  4% 35.0M/814M [00:00<00:26, 31.3MB/s]',
 '  5% 42.0M/814M [00:00<00:24, 32.8MB/s]',
 '  7% 57.0M/814M [00:01<00:19, 41.5MB/s]',
 '  9% 73.0M/814M [00:01<00:19, 40.5MB/s]',
 ' 13% 105M/814M [00:01<00:15, 48.4MB/s] ',
 ' 15% 121M/814M [00:01<00:12, 60.1MB/s]',
 ' 17% 137M/814M [00:02<00:10, 65.0MB/s]',
 ' 19% 153M/814M [00:02<00:13, 52.7MB/s]',
 ' 21% 172M/814M [00:02<00:09, 67.5MB/s]',
 ' 23% 185M/814M [00:03<00:10, 62.3MB/s]',
 ' 25% 201M/814M [00:03<00:10, 59.4MB/s]',
 ' 28% 225M/814M [00:03<00:08, 74.7MB/s]',
 ' 29% 236M/814M [00:03<00:11, 52.5MB/s]',
 ' 32% 257M/814M [00:03<00:08, 65.3MB/s]',
 ' 33% 268M/814M [00

In [11]:
from zipfile import ZipFile
fname='dogs-vs-cats-redux-kernels-edition.zip'
with ZipFile(fname,'r') as zip:
  zip.extractall()
  print('done')
# main zip file extract karva

done


In [12]:

from zipfile import ZipFile
fname='train.zip'
with ZipFile(fname,'r') as zip:
  zip.extractall()
  print('done')

done


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,BatchNormalization,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
%matplotlib inline

In [15]:
#change the name of train dir to dogs-vs -cats in folders
os.chdir('dogs-vs-cats')
if os.path.isdir('train/dog') is False:
  os.makedirs('train/dog')
  os.makedirs('train/cat')
  os.makedirs('valid/dog')
  os.makedirs('valid/cat')
  os.makedirs('test/dog')
  os.makedirs('test/cat')

  for c in random.sample(glob.glob('cat*'),1000):
    shutil.move(c,'train/cat')
  for c in random.sample(glob.glob('dog*'),1000):
    shutil.move(c,'train/dog')
  for c in random.sample(glob.glob('cat*'),200):
    shutil.move(c,'valid/cat')
  for c in random.sample(glob.glob('dog*'),200):
    shutil.move(c,'valid/dog')
  for c in random.sample(glob.glob('cat*'),100):
    shutil.move(c,'test/cat')
  for c in random.sample(glob.glob('dog*'),100):
    shutil.move(c,'test/dog')
  

In [16]:
os.chdir('../')

In [17]:
train_path='dogs-vs-cats/train'
valid_path='dogs-vs-cats/valid'
test_path='dogs-vs-cats/test'

In [18]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
  .flow_from_directory(directory=train_path,target_size=(224,224),classes=['cat','dog'],batch_size=10)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
  .flow_from_directory(directory=valid_path,target_size=(224,224),classes=['cat','dog'],batch_size=10)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
  .flow_from_directory(directory=test_path,target_size=(224,224),classes=['cat','dog'],batch_size=10,shuffle=False)

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [19]:
 imgs,labels=next(train_batches)

In [20]:
vgg16_model=tf.keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 7s 0us/step


In [22]:
model=Sequential()
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

   

In [23]:
for layer in model.layers:
  layer.trainable=False

In [24]:
model.add(Dense(units=2,activation='softmax'))

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=10,verbose=2)

Epoch 1/10
200/200 - 10s - loss: 0.2203 - accuracy: 0.9060 - val_loss: 0.1334 - val_accuracy: 0.9500
Epoch 2/10
200/200 - 10s - loss: 0.0611 - accuracy: 0.9755 - val_loss: 0.1048 - val_accuracy: 0.9600
Epoch 3/10
200/200 - 10s - loss: 0.0412 - accuracy: 0.9870 - val_loss: 0.1080 - val_accuracy: 0.9600
Epoch 4/10
200/200 - 10s - loss: 0.0321 - accuracy: 0.9925 - val_loss: 0.1048 - val_accuracy: 0.9650
Epoch 5/10
200/200 - 10s - loss: 0.0251 - accuracy: 0.9955 - val_loss: 0.1002 - val_accuracy: 0.9625
Epoch 6/10
200/200 - 10s - loss: 0.0211 - accuracy: 0.9945 - val_loss: 0.1051 - val_accuracy: 0.9625
Epoch 7/10
200/200 - 10s - loss: 0.0170 - accuracy: 0.9980 - val_loss: 0.1006 - val_accuracy: 0.9650
Epoch 8/10
200/200 - 10s - loss: 0.0143 - accuracy: 0.9980 - val_loss: 0.1024 - val_accuracy: 0.9650
Epoch 9/10
200/200 - 10s - loss: 0.0115 - accuracy: 0.9980 - val_loss: 0.1048 - val_accuracy: 0.9650
Epoch 10/10
200/200 - 10s - loss: 0.0095 - accuracy: 0.9980 - val_loss: 0.1040 - val_accura

In [27]:
predictions=model.predict(x=test_batches,verbose=2)

20/20 - 1s


In [28]:
cm=confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions,axis=-1))

In [29]:
model.save('catvsdog.h5')

In [31]:
import shutil
shutil.copy2('catvsdog.h5','drive/My Drive/Kaggle_json')

'drive/My Drive/Kaggle_json/catvsdog.h5'

In [30]:
!ls

catvsdog.h5  dogs-vs-cats  drive  kaggle.json  sample_data
